In [43]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import ast

data = pd.read_csv('original.csv')
# Удаление столбцов в которых сложно/невозможно что-то получить
data = data.drop(['street', 'zipcode', 'mls-id', 'private pool', 'MlsId', 'propertyType', 'status', 'city'], axis=1)
# Две строки которые сложно обработать/не несут смысла
data.drop([330468, 377183], inplace = True)

# Функции для будущей обработки
def func_pool(x):
    if type(x) != int:
        x = x.replace(x, '1')
    return int(x)

def func_built(x):
    if x == 'No Data':
        x = 1980
    return x

def data_func1(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][0].get('factValue')

def data_func2(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][1].get('factValue')

def data_func3(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][2].get('factValue')

def data_func4(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][3].get('factValue')

def data_func5(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][4].get('factValue')

def data_func6(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][5].get('factValue')

def data_func7(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][6].get('factValue')

def data_func8(x):
    m = []
    a = ast.literal_eval(x)
    a = dict(a[0])
    k = a['data'].get('Distance')
    for i in k:
        h = re.findall(r'\d+',i)
        if len(h) == 0:
            continue
        if len(h) == 1:
            m.append(float(h[0]))
            continue
        d = h[0] + '.' + h[1]
        m.append(float(d))
    if len(m) == 0:
        return 0
    else:
        return min(m)
    
def state(x):
    if x in top:
        return 2
    if x in mid:
        return 1
    return 0
    
# Списки по рейтингу штатов
top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']

# states - 0-2 В зависимости от благополучия штата
# distance - дистанция до ближайшей школы
# built - год постройки
# rebuilt - год реновации
# heating - отопление
# cooling - кондиционер
# parking - паркинг
# lotsize - площадь участка
# price - цена за 1 sqft
data['states'] = data['state'].apply(state)
data['distance'] = data['schools'].apply(data_func8)
data['built'] = data['homeFacts'].apply(data_func1)
data['rebuilt'] = data['homeFacts'].apply(data_func2)
data['heating'] = data['homeFacts'].apply(data_func3)
data['cooling'] = data['homeFacts'].apply(data_func4)
data['parking'] = data['homeFacts'].apply(data_func5)
data['lotsize'] = data['homeFacts'].apply(data_func6)
data['price'] = data['homeFacts'].apply(data_func7)

# Эти столбцы обработаны и больше не нужны
data = data.drop(['homeFacts', 'schools', 'state'], axis = 1)

# Сохранение для следующей обработки
data.to_csv('data.csv', index=False)

# Код работает 4 минуты

In [44]:
data = pd.read_csv('data.csv')

# Функции для будущей обработки
def func_pool(x):
    if type(x) != int:
        x = x.replace(x, '1')
    return int(x)

def func_built(x):
    if x == 'No Data':
        x = 1980
    return x

def state(x):
    if x in top:
        return 2
    if x in mid:
        return 1
    return 0

# Замена NaN и обработка столбцов
data['PrivatePool'] = data['PrivatePool'].fillna(0).apply(func_pool)
data['fireplace'] = data['fireplace'].fillna(0).apply(func_pool)
data['heating'] = data['heating'].fillna(0).apply(func_pool)
data['cooling'] = data['cooling'].fillna(0).apply(func_pool)
data['parking'] = data['parking'].fillna(0).apply(func_pool)
data['built'] = data['built'].fillna(1980).apply(func_built)
data['rebuilt'] = data['rebuilt'].fillna(0)

# Заполнение NaN в столбце rebuilt(Скорее всего можно реализовать намного лучше)

for x in range(len(data)):
    if data['rebuilt'][x] == 0:
        data['rebuilt'][x] = data['built'][x]
        
# Расширение бинарных признаков
pool_dummies = pd.get_dummies(data["PrivatePool"], prefix="Pool", dummy_na=False)
data = pd.concat([data, pool_dummies], axis=1)
fire_dummies = pd.get_dummies(data["fireplace"], prefix="fire", dummy_na=False)
data = pd.concat([data, fire_dummies], axis=1)
heat_dummies = pd.get_dummies(data["heating"], prefix="heat", dummy_na=False)
data = pd.concat([data, heat_dummies], axis=1)
cool_dummies = pd.get_dummies(data["cooling"], prefix="cool", dummy_na=False)
data = pd.concat([data, cool_dummies], axis=1)
park_dummies = pd.get_dummies(data["parking"], prefix="park", dummy_na=False)
data = pd.concat([data, park_dummies], axis=1)


data = data.drop(['fireplace', 'PrivatePool', 'heating', 'cooling', 'parking'], axis = 1)
# Сохранение для следующей обработки
data.to_csv('predone.csv', index=False)

# Код работает 50 секунд

In [45]:
# Обработка признаков для тюнинга
data = pd.read_csv('predone.csv')

def func_cost(x):
    if type(x) != float:
        x = x.replace('$','')
        x = x.replace(',','')
        x = x.replace('+','')
        x = x.replace('/mo','')
        return int(x)

def func_stor(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if (len(x) > 1):
            x = x[0]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 1
    return int(x)

def func_area(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 100
    return int(x)

# Заполнение NaN и очистка данных
# Многие значение брал из головы, потом менял и смотрел изменение точности модели
data['target'] = data['target'].apply(func_cost).fillna(7500)
data['stories'] = data['stories'].fillna(0).apply(func_stor)
data['beds'] = data['beds'].fillna(0).apply(func_stor)
data['baths'] = data['baths'].fillna(1).apply(func_stor)
data['sqft'] = data['sqft'].fillna(1900).apply(func_area)
data['lotsize'] = data['lotsize'].fillna(1900).apply(func_area)
data['price'] = data['price'].fillna(100).apply(func_area)

# Приведение к числовому формату
data['target'] = pd.to_numeric(data['target'])
data['stories'] = pd.to_numeric(data['stories'])
data['lotsize'] = pd.to_numeric(data['lotsize'])
data['sqft'] = pd.to_numeric(data['sqft'])
data['beds'] = pd.to_numeric(data['beds'])
data['baths'] = pd.to_numeric(data['baths'])
data['price'] = pd.to_numeric(data['price'])
data['built'] = pd.to_numeric(data['built'])
data['rebuilt'] = pd.to_numeric(data['rebuilt'])

# Очистка от бредовых данных
data = data[data['baths'] < 60]
data = data[data['beds'] < 60]
data = data[data['stories'] < 100]
# Сохранение для обучения модели
data.to_csv('learning.csv', index=False)

# Код работает 7 секунд

In [46]:
data = pd.read_csv('learning.csv')

# Выбор признаков
X = data.drop('target', axis = 1)
y = data['target']

# Разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23, test_size=0.7)

# Пробовал многие алгоритмы для задачи регрессии - этот показала лучший результат
regr = RandomForestRegressor(n_estimators = 120, 
                             random_state = 23, 
                             max_depth = 11, 
                             n_jobs=-1, 
                             min_samples_split = 5, 
                             min_samples_leaf = 3)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
mse_train = r2_score(y_test, y_pred)
print('R2 = {}'.format(mse_train))
# R2 = 0.7712074758398859

# Код работает 45 секунд при n_jobs = 1
# Код работает 13 секунд при n_jobs = -1

R2 = 0.7712074758398859


In [108]:
#Результат в csv
submission = pd.DataFrame({
    "target": y_pred
})
submission.to_csv("submission.csv", index=False)